In [1]:
import pandas as pd
import numpy as np
import yaml

from scipy.signal import argrelextrema

from lightgbm import LGBMClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from category_encoders import CountEncoder, TargetEncoder

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import sys

sys.path.append('..')

from source.utils import read_data, handle_nans, preprocess

In [2]:
train,  = read_data(who=['train'])

In [3]:
train, = handle_nans(preprocess(train))

(                                          user_id  REGION         TENURE  \
 0        00000bfd7d50f01092811bc0c8d7b0d6fe7c3596  FATICK   K > 24 month   
 1        00000cb4a5d760de88fecb38e2f71b7bec52e834     NAN  I 18-21 month   
 2        00001654a9d9f96303d9969d0a4a851714a4bb57     NAN   K > 24 month   
 3        00001dd6fa45f7ba044bd5d84937be464ce78ac2   DAKAR   K > 24 month   
 4        000028d9e13a595abe061f9b58f3d76ab907850f   DAKAR   K > 24 month   
 ...                                           ...     ...            ...   
 2154043  ffffe85215ddc71a84f95af0afb0deeea90e6967     NAN   K > 24 month   
 2154044  ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15   THIES   K > 24 month   
 2154045  fffff172fda1b4bb38a95385951908bb92379809     NAN   K > 24 month   
 2154046  fffff5911296937a37f09a37a549da2e0dad6dbb   THIES   K > 24 month   
 2154047  fffff6dbff1508ea2bfe814e5ab2729ce6b788c2     NAN   K > 24 month   
 
          MONTANT  FREQUENCE_RECH  REVENUE  ARPU_SEGMENT  FREQUENCE  \
 0 

In [19]:
target = train['CHURN']
train['REGULARITY'] = train['REGULARITY'].astype('float')
train = train.drop(['user_id', 'MRG', 'CHURN'], axis=1)
# test = test.drop(['user_id', 'MRG'], axis=1)

In [8]:
# def get_bins(data):
#     x, y = sns.kdeplot(data[data>0], log_scale=True, color='b').get_lines()[0].get_data()
#     plt.close()
#     ext, = argrelextrema(y, np.less)
#     return [min(data.min(), x.min()) - 1, *x[ext], max(data.max(), x.max()) + 1]

# for col in tqdm(train.columns[train.dtypes=='float']):
#     bins = get_bins(train[col])
#     train[col + '_minimums'] = pd.cut(train[col], bins)
#     test[col + '_minimums'] = pd.cut(test[col], bins)

# ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
# ohe_train = ohe.fit_transform(train[train.columns[train.dtypes=='object']])
# ohe_test = ohe.transform(test[test.columns[test.dtypes=='object']])

# train = np.hstack([ohe_train, train[train.columns[train.dtypes!='object']].values])
# test = np.hstack([ohe_test, test[test.columns[test.dtypes!='object']].values])
# le = CountEncoder()
# le_train = le.fit_transform(train[train.columns[train.dtypes=='object']])
# le_test = le.transform(test[test.columns[test.dtypes=='object']])

# train = np.hstack([le_train, train[train.columns[train.dtypes!='object']].values])
# test = np.hstack([le_test, test[test.columns[test.dtypes!='object']].values])

In [24]:
X_train, X_val, y_train, y_val = train_test_split(train, target, random_state=42)

te = TargetEncoder()

le_train = le.fit_transform(X_train[X_train.columns[X_train.dtypes=='object']], y_train)
le_val = le.transform(X_val[X_val.columns[X_val.dtypes=='object']])

X_train = np.hstack([le_train, X_train[X_train.columns[X_train.dtypes!='object']].values])
X_val = np.hstack([le_val, X_val[X_val.columns[X_val.dtypes!='object']].values])

c:\users\ivan\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [27]:
lr = RidgeClassifier(class_weight='balanced')
lr.fit(X_train, y_train)

roc_auc_score(y_val, lr.decision_function(X_val))

0.9189821997834796

In [30]:
lr = RidgeClassifier(class_weight='balanced')
lr.fit(X_train, y_train)

roc_auc_score(y_val, lr.decision_function(X_val))

0.9250693117958613

In [49]:
lr = RidgeClassifier(class_weight='balanced')
lr.fit(train, target)

prediction = lr.decision_function(test)

In [53]:
train = np.hstack([train, lr.decision_function(train).reshape(-1, 1)])
test = np.hstack([test, lr.decision_function(test).reshape(-1, 1)])

In [54]:
X_train, X_val, y_train, y_val = train_test_split(train, target)

In [30]:
lgbm = LGBMClassifier()

lgbm.fit(np.log1p(X_train), y_train)
roc_auc_score(y_val, lgbm.predict_proba(np.log1p(X_val))[:, 1])

0.93114810219399

In [51]:
make_submission(prediction, 'ridge_nans')

In [ ]:

# mask = np.random.choice(X_train.index, size=100000, replace=False)
# knn = KNeighborsClassifier()
# knn.fit(X_train.loc[mask], y_train.loc[mask])

# roc_auc_score(y_test, knn.predict_proba(X_test))

In [36]:
# quantiles = [0, *np.arange(0.05, 1, 0.1), 1]
# bins = train.MONTANT.quantile(quantiles)
# train.groupby(pd.cut(train.MONTANT, bins, duplicates='drop', include_lowest=True)).CHURN.mean()

MONTANT
(9.999, 500.0]         0.127763
(500.0, 1000.0]        0.079491
(1000.0, 2000.0]       0.055419
(2000.0, 3000.0]       0.378253
(3000.0, 4600.0]       0.030392
(4600.0, 7900.0]       0.022264
(7900.0, 15350.0]      0.014272
(15350.0, 470000.0]    0.010045
Name: CHURN, dtype: float64

In [68]:
positive = target.sum()
negative = target.shape[0] - positive
need = negative - positive

new_idx = np.hstack([np.random.choice(target[target==1].index, need), target[target==1].index])

X_oversampled = pd.get_dummies(pd.concat([train[target==0], train.loc[new_idx]]))
y_oversampled = pd.concat([target[target==0], target.loc[new_idx]])

In [69]:
idx = np.random.choice(target[target==1].index, int(positive * 0.3), replace=False)

In [70]:
test_idx1 = np.random.choice(target[target==1].index, int(positive * 0.3), replace=False)
test_idx0 = np.random.choice(target[target==0].index, int(negative * 0.3), replace=False)
full_test_idx = np.hstack([test_idx0, test_idx1])
full_train_idx = np.setdiff1d(target.index, full_test_idx)

X_train, X_test, y_train, y_test = X_oversampled.loc[full_train_idx], X_oversampled.loc[full_test_idx],\
                                    y_oversampled.loc[full_train_idx], y_oversampled.loc[full_test_idx]

In [71]:
lr = RidgeClassifier()

lr.fit(X_train, y_train)
roc_auc_score(y_test, lr.decision_function(X_test))

0.9254100729100593

In [42]:
estimators = [
    ('lgbm1', LGBMClassifier(random_state=1)),
    ('lgbm2', LGBMClassifier(random_state=2)),
    ('lgbm3', LGBMClassifier(random_state=3))
 ]

stacking = StackingClassifier(
    estimators=estimators, final_estimator=RidgeClassifier()
)

In [43]:
stacking.fit(X_train, y_train)

StackingClassifier(estimators=[('lgbm1', LGBMClassifier(random_state=1)),
                               ('lgbm2', LGBMClassifier(random_state=2)),
                               ('lgbm3', LGBMClassifier(random_state=3))],
                   final_estimator=RidgeClassifier())

In [46]:
roc_auc_score(y_val, stacking.decision_function(X_val))

0.9308577263466179

In [28]:
mapper = {x[1] : x[0] + 1 for x in enumerate(
    [x[0] for x in sorted(train.TOP_PACK.value_counts().items(), key=lambda x: -x[1])]
)}
quantiles = [0, *np.arange(0.05, 1, 0.1), 1]
bins = train.TOP_PACK.map(mapper).quantile(quantiles)

quantiled = pd.cut(train.TOP_PACK.map(mapper), bins, duplicates='drop')

train.groupby(train.TOP_PACK.map(quantiled)).CHURN.mean()

TOP_PACK
(1.0, 2.0]      NaN
(2.0, 3.0]      NaN
(3.0, 5.0]      NaN
(5.0, 7.0]      NaN
(7.0, 9.0]      NaN
(9.0, 13.0]     NaN
(13.0, 21.0]    NaN
(21.0, 140.0]   NaN
Name: CHURN, dtype: float64